# AffectNet to Tensorflow (Keras)

This preparation step will convert the AffectNet Database to the Keras directory structure containing the classes as directories.

All images are 224x224.

## Class Definitions

0: Neutral,
1: Happiness,
2: Sadness,
3: Surprise,
4: Fear,
5: Disgust,
6: Anger,
7: Contempt,
8: None,
9: Uncertain,
10: No-Face

## Input

```
train_set
└───annotations
│   │   100000_aro.npy (Arousal -1 - 1)
│   │   100000_exp.npy (Expression 0 - 10)
│   │   100000_lnd.npy (68 Facial Landmark Points)
│   │   100000_val.npy (Valence -1 - 1)
└───images
│   │   100000.jpg
```

## Expected Output

```
train_set
└───class001
│   │   100000.jpg
```

In [1]:
import numpy as np
from pathlib import Path
import os
import shutil
import glob

In [8]:
pwd = os.getcwd()
dataset = f"/Users/luca/Downloads/val_set"
target_dir = f"{dataset}_keras"
classes = ["0", "1", "2", "3", "4", "5", "6", "7"]
number_images_to_process_per_class = 500  # 0 for all images

In [9]:
if not Path(dataset + "/images").is_dir():
    raise NotADirectoryError("images dir not found")

if not Path(dataset + "/annotations").is_dir():
    raise NotADirectoryError("annotations dir not found")

# ensure target dir and throw if it already exists
if Path(target_dir).is_dir():
    raise FileExistsError("Target dir already exists. Exiting")

Path(target_dir).mkdir(parents=True, exist_ok=True)


In [10]:
annotations_dir = dataset + "/annotations"

# load filenames of all expressions
expressions = glob.glob(f"{annotations_dir}/*_exp.npy", recursive=False)
# empty dictionary with classes as keys and filenames as values
classified_images = {key: list() for key in classes}

print("working through a total of " + str(len(expressions)) + " images")

# iterate filenames of expressions
for (i, image_expression_filename) in enumerate(expressions):

    if i % 100 == 0:
        # print percentage of expression progress
        print('Progress: {:.2f}'.format(i / len(expressions) * 100))

    # load from numpy file
    image_expression = np.load(image_expression_filename)
    # get first value (there is only one in this case)
    image_expression_val = image_expression.flat[0]
    # if it's not one of the defined discrete expressions abort here
    if int(image_expression_val) > 7:
        continue
    # add to the dictionary
    # {
    #   "0": ["filename1", "filename2", ...],
    #   "1": ["filename3", "filename4", ...],
    # }

    if len(classified_images[image_expression_val]) < number_images_to_process_per_class:
        classified_images[image_expression_val].append(
            image_expression_filename)


working through a total of 3999 images
Progress: 0.00
Progress: 2.50
Progress: 5.00
Progress: 7.50
Progress: 10.00
Progress: 12.50
Progress: 15.00
Progress: 17.50
Progress: 20.01
Progress: 22.51
Progress: 25.01
Progress: 27.51
Progress: 30.01
Progress: 32.51
Progress: 35.01
Progress: 37.51
Progress: 40.01
Progress: 42.51
Progress: 45.01
Progress: 47.51
Progress: 50.01
Progress: 52.51
Progress: 55.01
Progress: 57.51
Progress: 60.02
Progress: 62.52
Progress: 65.02
Progress: 67.52
Progress: 70.02
Progress: 72.52
Progress: 75.02
Progress: 77.52
Progress: 80.02
Progress: 82.52
Progress: 85.02
Progress: 87.52
Progress: 90.02
Progress: 92.52
Progress: 95.02
Progress: 97.52


In [11]:
# iterate all classes (0-7)
for class_key in classified_images.keys():
    # /abc/val_set_keras/0
    target_dir_class = target_dir + "/" + class_key
    Path(target_dir_class).mkdir(parents=True, exist_ok=True)
    # iterate expressions per class
    # expression_image_file = /abc/val_set/annotations/1234_exp.npy
    for expression_image_file in classified_images[class_key]:
        # image_filename = /abc/val_set/annotations/1234.jpg
        image_filename = expression_image_file[:-8] + ".jpg"
        # image_filename = /abc/val_set/images/1234.jpg
        image_filename = image_filename.replace("/annotations/", "/images/")
        # target_filename = 1234
        target_filename = Path(image_filename).stem
        # target_file = /abc/val_set_keras/0/1234.jpg
        target_file = f'{target_dir}/{class_key}/{target_filename}.jpg'
        shutil.copy(image_filename, target_file)
